This notebook shows how I generated lightmap positions. It is most definitely not the easiest or the best way to do it. We'll use the `trimesh` library to load the mesh and generate the lightmap positions. The lightmap is generated by:

1. Loading the STL mesh
2. Voxelizing it using `mesh.voxelized(pitch)`
3. Create a copy of the voxelized mesh and fill it using `mesh.fill(method)`
4. Get the lightmap positions by subtracting the set of unfilled voxels from the filled voxels.

In [55]:
import trimesh
import numpy as np

tpc = trimesh.load_mesh(
    "/home/sam/sw/chroma-lxe/data/stl/EA-HV_no_electrode_plugged.STL"
)

pitch = 5 # mm
voxels = tpc.voxelized(pitch)
filled = voxels.copy().fill(method="holes")

f_p = filled.points
v_p = voxels.points

f_p_ = [tuple(p) for p in f_p]
v_p_ = [tuple(p) for p in v_p]

inside = np.array(list(set(f_p_) - set(v_p_)))

In [ ]:
# hopefully this looks good!

def plot_generated_points(all_points, inside_points):
    pc_fp = trimesh.PointCloud(all_points)
    pc_fp.colors = np.array([255, 0, 0, 255])     # all points: red
    pc_inside = trimesh.PointCloud(inside_points)
    pc_inside.colors = np.array([0, 255, 0, 255]) # inside points: green

    scene = trimesh.Scene([pc_fp, pc_inside])
    return scene

plot_generated_points(f_p, inside).show()

Not great. The fill algorithm filled in the space between the edges of the flanges. We can try another fill method, `orthographic` and see what it gives us.

In [56]:
filled = voxels.copy().fill(method="orthographic")
f_p = filled.points
v_p = voxels.points

f_p_ = [tuple(p) for p in f_p]
v_p_ = [tuple(p) for p in v_p]

inside = np.array(list(set(f_p_) - set(v_p_)))

In [ ]:
plot_generated_points(f_p, inside).show()

This is much better, since the TPC volume is now totally filled. Unfortunately every gap in the entire system is filled, so we'll need to refine this by removing the filled voxels that are not connected to the TPC volume. We can do this by creating a graph of the filled voxels and removing the connected components that are not connected to the TPC volume.

This takes ~10 sec to do.

In [63]:
import networkx as nx
from tqdm import tqdm
from scipy.spatial import cKDTree

# create a graph and add every position as nodes
G = nx.Graph()
for i, p in enumerate(inside):
    G.add_node(i, pos=tuple(p))

adjacency_offsets = [
    (pitch, 0, 0),
    (-pitch, 0, 0),
    (0, pitch, 0),
    (0, -pitch, 0),
    (0, 0, pitch),
    (0, 0, -pitch),
]

# create a KDTree for fast neighbor search
kdtree = cKDTree(inside)

# for each position, find its neighbors and add an
# edge between them if they are next to each other
for i, position in enumerate(tqdm(inside)):
    for offset in adjacency_offsets:
        neighbor_position = position + np.array(offset)
        distances, indices = kdtree.query(neighbor_position, k=1)
        if distances < pitch:
            neighbor_index = indices
            G.add_edge(i, neighbor_index)

# finally, get the largest connected component and its positions
largest_cc = max(nx.connected_components(G), key=len)
largest_cluster_positions = np.array([G.nodes[node]["pos"] for node in largest_cc])

100%|██████████| 44390/44390 [00:10<00:00, 4412.23it/s]


In [ ]:
plot_generated_points(inside, largest_cluster_positions).show()

Perfect! Now we can save them:

In [ ]:
np.save("../data/lightmap_positions.npy", largest_cluster_positions)